# Master Thesis - Quintine Sol

In [ ]:
import parselmouth
import random
import pandas as pd
import os.path
import matplotlib.pyplot as plt
import numpy as np
import whisper
import re
import warnings
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.preprocessing import StandardScaler
from tqdm.notebook import tqdm

In [ ]:
plt.rcParams.update({'font.size': 12}) # Set default font size for plots

## Load Data

### Intake Session

In [ ]:
# Load the intake questionnaires
questionnaire_intake = pd.read_csv("./data/Intake_session/cleaned_curated_Intake_Session.csv")
questionnaire_intake.head()

### Conversation Setting

In [ ]:
# Load the conversation questionnaires
questionnaires_conv = pd.read_csv("./data/Conversation/Evaluation_Questionnaires/cleaned_curated_conv_coord_data_combined_ffinal.csv")
print("Conversation questionnaires:", questionnaires_conv.shape)

### Collaboration Setting

In [ ]:
# Load the collaboration questionnaires
questionnaires_coll = pd.read_csv("./data/Collaboration/Evaluation_Questionnaires/cleaned_curated_imp_coll_coord_data_combined_ffinal.csv")
print("Collaboration questionnaires:", questionnaires_coll.shape)

## Create Subset

### Filter: Batches

Available batches.

In [ ]:
# Set of batch numbers
batches = set(range(1, 68))

# Set of missing batch numbers (in collaboration setting)
missing_batches = {1, 2, 3, 4, 5, 7, 26, 34, 36, 44, 50, 51, 52, 62}

# Set of available batch numbers
available_batches = list(batches - missing_batches)

Randomly select 30 batches.

In [ ]:
no_batches = 30 # Number of batches to select 

# Seed random number generator
random.seed(42)

# Randomly select no_batches from the set of available batches and sort them
selected_batches_numbers = sorted(random.sample(available_batches, no_batches))

print(selected_batches_numbers) 

Convert numbers to strings.

In [ ]:
# Convert batch numbers to strings
selected_batches = ["Batch_" + str(batch) for batch in selected_batches_numbers]

# Print (sorted) selected batches
print(selected_batches)

Filter questionnaire data based on Batch ID.

In [ ]:
# Filter conversation questionnaires
filtered_questionnaires_conv = questionnaires_conv[questionnaires_conv["batchID"].isin(selected_batches)]
print("Conversation questionnaires filtered:", filtered_questionnaires_conv.shape)

# Filter collaboration questionnaires
filtered_questionnaires_coll = questionnaires_coll[questionnaires_coll["batchID"].isin(selected_batches)]
print("Collaboration questionnaires filtered:", filtered_questionnaires_coll.shape)

### Filter: Dyads with Both Participants Evaluating Both Settings

In [ ]:
def reciprocal_dyadID(df):
    """
    Get unique dyadIDs where both participants have evaluated each other.

    Args:
    - df (pd.DataFrame): The dataframe containing the (conv or coll) questionnaire data.

    Returns:
    - list: A list containing unique dyadIDs where both participants have evaluated each other.
    """
    # Get unique triplets of selfPID, otherPID, and dyadID occurring in the dataframe
    unique_triplets = set(zip(df["selfPID"], df["otherPID"], df["dyadID"]))

    # Check if the evaluations are reciprocal for each dyad
    reciprocal_dyads = [dyadID for (selfPID, otherPID, dyadID) in unique_triplets if (otherPID, selfPID, dyadID) in unique_triplets]
    
    # Remove duplicates
    reciprocal_dyads = set(reciprocal_dyads)
    
    return reciprocal_dyads

In [ ]:
# Get the unique dyadIDs where both participants have evaluated each other in both settings
reciprocal_dyads = reciprocal_dyadID(filtered_questionnaires_conv) & reciprocal_dyadID(filtered_questionnaires_coll)

# Only contain rows with reciprocal dyadIDs
filtered_reciprocal_conv = filtered_questionnaires_conv[filtered_questionnaires_conv["dyadID"].isin(reciprocal_dyads)]
filtered_reciprocal_coll = filtered_questionnaires_coll[filtered_questionnaires_coll["dyadID"].isin(reciprocal_dyads)]

# Print the number of rows in the filtered dataframes
print("Conversation questionnaires filtered on reciprocity:", filtered_reciprocal_conv.shape)
print("Collaboration questionnaires filtered on reciprocity:", filtered_reciprocal_coll.shape)

### Filter: Dyads with Available Recordings (at least 2 minutes) for Both Participants in Both Settings

Find available recording (at least 2 minutes) for each participant in each dyad.

In [ ]:
def available_file(recording_file, setting, batch_id):
    """
    Check if the specified recording file is available in the data folder.
    
    Args:
    - recording_file (str): The name of the recording file.
    - setting (str): The setting of the recording file (i.e., "conv" or "coll").
    - batch_id (str): The batchID value of the recording file (e.g. "Batch_8").

    Returns:
    - bool: True if the file is available, False otherwise.
    """
    # Define the file path based on batch_id and setting
    if setting == "conv":
        file_path = f"data/Conversation/WAV_recordings/{batch_id}/{recording_file}.wav"
    elif setting == "coll":
        file_path = f"data/Collaboration/WAV_recordings/{batch_id}/{recording_file}.wav"
    else:
        raise ValueError("Invalid setting. Please specify 'conv' or 'coll'.")

    # Check if file is present in data folder
    return os.path.isfile(file_path)

In [ ]:
def available_recording(row, setting, min_length=120):
    """
    Get the available recording of the specified participant in the specified setting (self_local if available, otherwise self_remote).
    If duration of self_local recording is less than 2 minutes or the file is unavailable, use self_remote recording.
    If duration of self_remote recording is also less than 2 minutes or the file is also unavailable, return None.

    Args:
    - row (pd.Series): The row of the evaluation questionnaire dataframe.
    - setting (str): The setting of the questionnaire data (i.e., "conv" or "coll").
    - min_length (int): The minimum length of the recording in seconds (default is 120 seconds).

    Returns:
    - str: The available recording of the specified participant in the specified setting.
    """
    # Check if the self_local recording is available and meets the minimum length requirement
    if row[f"Availability_{setting}_rec_self_local"] == 1 and row[f"duration_{setting}_rec_self_local"] >= min_length and available_file(row[f"{setting}_rec_self_local"], setting, row["batchID"]):
        return row[f"{setting}_rec_self_local"]
    # Check if the self_remote recording is available and meets the minimum length requirement
    elif row[f"Availability_{setting}_rec_self_remote"] == 1 and row[f"duration_{setting}_rec_self_remote"] >= min_length and available_file(row[f"{setting}_rec_self_remote"], setting, row["batchID"]):
        return row[f"{setting}_rec_self_remote"]
    # Otherwise, return None
    else:
        return None

In [ ]:
def create_participant_recordings(df, setting):
    """
    Add a new column to the dataframe with the available recording for the specified participant in the specified setting.
    
    Args:
    - df (pd.DataFrame): The dataframe containing the questionnaire data.
    - setting (str): The setting of the questionnaire data (i.e., "conv" or "coll").

    Returns:
    - pd.DataFrame: The filtered dataframe with available recordings.
    """
    # Create a copy of the dataframe to avoid modifying the original
    df = df.copy()

    # Get the available recording for each participant in the specified setting
    df["recording_file"] = df.apply(lambda row: available_recording(row, setting), axis=1)

    return df

In [ ]:
# Create dataframe with available participant recordings for conversation setting
recordings_conv = create_participant_recordings(filtered_reciprocal_conv, "conv")

# Create dataframe with available participant recordings for collaboration setting
recordings_coll = create_participant_recordings(filtered_reciprocal_coll, "coll")

Only keep dyads with available recordings for both participants.

In [ ]:
def missing_recordings_dyadID(df):
    """
    Get the unique dyadID values where recording is unavailable in the questionnaire data.

    Args:
    - df (pd.DataFrame): The dataframe containing the (conv or coll) questionnaire data including available participant recordings.

    Returns:
    - set: The set of dyadIDs with missing recordings in the questionnaire data.
    """
    # Find dyadIDs with missing recordings
    missing_rec = df[df["recording_file"].isnull()]["dyadID"].unique()

    # Remove duplicates
    return set(missing_rec)

In [ ]:
# Find dyadIDs with missing recordings in the conversation or collaboration setting
missing_dyads = missing_recordings_dyadID(recordings_conv) | missing_recordings_dyadID(recordings_coll)

# Show the number of dyads with missing recordings in one (or both) settings
print(f"There are {len(missing_dyads)} dyads with missing recordings in one (or both) settings: {missing_dyads}.")

# Remove these dyads from the participant recordings
filtered_recordings_conv = recordings_conv[~recordings_conv["dyadID"].isin(missing_dyads)]
filtered_recordings_coll = recordings_coll[~recordings_coll["dyadID"].isin(missing_dyads)]

# Print the number of rows in the filtered dataframes
print("Conversation recordings filtered on missing recordings:", filtered_recordings_conv.shape)
print("Collaboration recordings filtered on missing recordings:", filtered_recordings_coll.shape)

### Figure of Filtering Steps

In [ ]:
# The number of observations before/after each filtering step
conv_steps = [questionnaires_conv.shape[0], filtered_questionnaires_conv.shape[0], filtered_reciprocal_conv.shape[0], filtered_recordings_conv.shape[0]]
coll_steps = [questionnaires_coll.shape[0], filtered_questionnaires_coll.shape[0], filtered_reciprocal_coll.shape[0], filtered_recordings_coll.shape[0]]

# The filtering steps
names_steps = ["Initial", "Batches", "Reciprocity", "Recordings"]

x = np.arange(len(names_steps))  # the label locations
width = 0.35  # width of the bars

# Create a bar plot
fig, ax = plt.subplots(figsize=(10, 6))
bars1 = ax.bar(x - width/2, conv_steps, width, label='Conversation', color='limegreen')
bars2 = ax.bar(x + width/2, coll_steps, width, label='Collaboration', color='orange')

# Add exact values on top of each bar
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 5),
                    textcoords="offset points",
                    ha='center', va='bottom')

# Labels and titles
ax.set_xlabel('Filtering Step')
ax.set_ylabel('Number of Observations (max 2 per dyad)')
ax.set_title('Remaining Observations after each Filtering Step')
ax.set_xticks(x)
ax.set_xticklabels(names_steps)
ax.legend()
ax.grid(axis='y', linestyle='--', alpha=0.7)

fig.tight_layout()
plt.show()

### Retain only Columns of Interest

In [ ]:
final_df_conv = filtered_recordings_conv[["selfPID", "otherPID", "dyadID", "batchID", "conv_CC_rapp1", "recording_file"]]
final_df_coll = filtered_recordings_coll[["selfPID", "otherPID", "dyadID", "batchID", "coll_CC_rapp1", "recording_file"]]

print(f"Number of rows in the dataframes: {final_df_conv.shape[0]} and {final_df_coll.shape[0]}.")

### Number of Participants

In [ ]:
# Count number of unique selfPID
unique_selfPID_conv = final_df_conv["selfPID"].nunique()
unique_selfPID_coll = final_df_coll["selfPID"].nunique()

# Count number of participants in conversation and collaboration
print(f"Number of participants in conversation: {unique_selfPID_conv}")
print(f"Number of participants in collaboration: {unique_selfPID_coll}")

### Demographics

In [ ]:
# Filter the intake questionnaire to only include participants in the final (conv or coll) dataframes
subset_questionnaire_intake = questionnaire_intake[questionnaire_intake["globalPID"].isin(final_df_conv["selfPID"])]

Age.

In [ ]:
# Age of participants in the subset
age_participants = subset_questionnaire_intake["Age_part"]

# Mean
mean_age = age_participants.mean()
print(f"Mean age of participants: {mean_age}")

# Standard deviation
std_age = age_participants.std()
print(f"Standard deviation of age of participants: {std_age}")

Gender.

In [ ]:
# Gender of participants in the subset
gender_participants = subset_questionnaire_intake["Sex_part"]

# Count females
females = gender_participants[gender_participants == "Female"].count()
print(f"Number of females: {females}")

Nationalities.

In [ ]:
# Distribution of nationalities in the original dataset
original_nationalities = questionnaire_intake["Nat_part"]
print(f"Number of nationalities: {len(original_nationalities.unique())}")
print(f"Nationalities: {original_nationalities.value_counts()}")

# Distribution of nationalities in the subset
nationality_participants = subset_questionnaire_intake["Nat_part"]
print(f"Number of nationalities: {len(nationality_participants.unique())}")
print(f"Nationalities: {nationality_participants.value_counts()}")

# Nationalities not present in the subset that were present in the complete dataset
print(f"Missing nationalities: {set(original_nationalities.unique()) - set(nationality_participants.unique())}")

### Distribution of Rapport Ratings

In [ ]:
# The number of rapport ratings per level
conv_counts = final_df_conv["conv_CC_rapp1"].value_counts().sort_index()
coll_counts = final_df_coll["coll_CC_rapp1"].value_counts().sort_index()

# Custom x-axis labels
x_labels = [1, 2, 3, 4, 5]

# Position of the bars
x = np.arange(len(x_labels))  # x locations for the groups
width = 0.35  # width of the bars

# Create a figure and axis
fig, ax = plt.subplots(figsize=(10, 6))

# Create bars for Conversation and Collaboration settings
bars1 = ax.bar(x - width/2, conv_counts, width, label='Conversation', color='limegreen')
bars2 = ax.bar(x + width/2, coll_counts, width, label='Collaboration', color='orange')

# Add exact values on top of each bar
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 5),
                    textcoords="offset points",
                    ha='center', va='bottom')

# Labels and title
ax.set_xlabel("Rapport Ratings")
ax.set_ylabel("Frequency")
ax.set_title("Rapport Ratings in Conversation and Collaboration Settings")

# Customize the x-axis tick labels
ax.set_xticks(x)
ax.set_xticklabels(x_labels)

# Add a legend
ax.legend()

# Add gridlines for better readability
ax.grid(axis='y', linestyle='--', alpha=0.7)

# Adjust the layout to prevent clipping
fig.tight_layout()

# Show the plot
plt.show()

### Combine Data of each Dyad into One Row 

In [ ]:
def combine_dyad_rows(df, setting):
    """
    Merge the dataframe with itself to find reciprocal dyad rows and combine them.
    In other words, this would result in two times less rows.

    Args:
    - df (pd.DataFrame): The dataframe containing the questionnaire data.
    - setting (str): The setting of the questionnaire data (i.e., "conv" or "coll").

    Returns:
    - pd.DataFrame: The combined dataframe.
    """
    # Merge the dataframe with itself to find reciprocal rows
    merged_df = df.merge(df,
        left_on=["selfPID", "otherPID", "dyadID", "batchID"],
        right_on=["otherPID", "selfPID", "dyadID", "batchID"],
        suffixes=("_self", "_other")
    )

    # Select and rename the desired columns
    df_combined = merged_df[[
        "selfPID_self", "otherPID_self", "dyadID", "batchID",
        f"{setting}_CC_rapp1_self", f"{setting}_CC_rapp1_other",
        "recording_file_self", "recording_file_other"
    ]].rename(columns={
        "selfPID_self": "selfPID",
        "otherPID_self": "otherPID",
        f"{setting}_CC_rapp1_self": f"self_{setting}_CC_rapp1",
        f"{setting}_CC_rapp1_other": f"other_{setting}_CC_rapp1",
        "recording_file_self": "self_recording_file",
        "recording_file_other": "other_recording_file"
    })

    # Drop duplicate rows based on dyadID
    df_combined = df_combined.drop_duplicates(subset=["dyadID"])

    return df_combined

In [ ]:
# Combine dyad rows in the conversation dataframe
final_df_combined_conv = combine_dyad_rows(final_df_conv, "conv")

# Combine dyad rows in the collboartion dataframe
final_df_combined_coll = combine_dyad_rows(final_df_coll, "coll")

print(f"Number of rows in the combined dataframes: {final_df_combined_conv.shape[0]} and {final_df_combined_coll.shape[0]}.")

## Conversation Dynamics

### Extract Time and Intensity Values (from trimmed sound)

In [ ]:
def extract_sound(recording_file, setting, batch_id):
    """
    Extract the audio sound from the specified recording file.

    Args:
    - recording_file (str): The name of the recording file.
    - setting (str): The setting of the recording file (i.e., "conv" or "coll").
    - batch_id (str): The batchID value of the recording file (e.g. "Batch_8").

    Returns:
    - parselmouth.Sound: The audio sound from the recording file.
    """
    # Define the file path based on batch_id and setting
    if setting == "conv":
        file_path = f"data/Conversation/WAV_recordings/{batch_id}/{recording_file}.wav"
    elif setting == "coll":
        file_path = f"data/Collaboration/WAV_recordings/{batch_id}/{recording_file}.wav"
    else:
        raise ValueError("Invalid setting. Please specify 'conv' or 'coll'.")

    # Load the recording file
    sound = parselmouth.Sound(file_path)
    
    return sound

In [ ]:
def trim(sound, duration=180):
    """
    Trim the sound to the specified duration (default is 180 seconds).

    Args:
    - sound (parselmouth.Sound): The audio data from the recording file.
    - duration (int): The duration to trim the recording in seconds (default is 180 seconds).

    Returns:
    - parselmouth.Sound: The trimmed sound.
    - float: The new duration of the sound in seconds.
    """
    # Get the duration of the recording
    duration_sound = sound.get_total_duration()

    # Trim the recording to the specified duration
    if duration_sound > duration:
        sound = sound.extract_part(from_time=0, to_time=duration)

    new_duration = sound.get_total_duration()

    return sound, new_duration

In [ ]:
def extract_time_intensity(sound):
    """
    Extract time and intensity values from the sound.

    Args:
    - sound (parselmouth.Sound): The audio sound.

    Returns:
    - np.array: The time values.
    - np.array: The intensity values.
    """
    # Convert the sound to intensity
    intensity = sound.to_intensity()

    # Extract time and intensity values
    time_values = intensity.xs()
    intensity_values = intensity.values.T
    
    return time_values, intensity_values

### Extract Turns

In [ ]:
def get_turns(time_values, intensity_values, threshold=50, max_pause_duration=0.2):
    """
    Extract turns from the time and intensity values.

    Args:
    - time_values (np.array): The time values.
    - intensity_values (np.array): The intensity values.
    - threshold (int): The intensity threshold for detecting voiced frames (default is 50).
    - max_pause_duration (float): The maximum allowed pause duration within turns (default is 0.2 seconds).
    
    Returns:
    - list of tuples: [(start_time, end_time), ...] representing the turns.
    """
    # Identify voiced frames where intensity exceeds the thresholds
    voiced = intensity_values >= threshold

    # Initialize variables
    turns = [] # List to store turns
    start = None # Start of a turn
    last_voiced = None # Last voiced frame
    
    # Find consective voiced frames
    for t in range(len(time_values)):
        if voiced[t] and start is None: # Start of a turn
            start = time_values[t]
            last_voiced = t
        elif voiced[t] and start is not None: # Continue a turn
            last_voiced = t
        elif not voiced[t] and start is not None and (time_values[t] - time_values[last_voiced]) >= max_pause_duration: # End of a turn
            if time_values[last_voiced] - start > 0: # Only add if the turn is longer than 0 seconds
                turns.append((start, time_values[last_voiced])) # Turn: start until last voiced

            # Reset start and last_voiced
            start = None
            last_voiced = None

    # End the last started turn
    if start is not None:
        turns.append((start, time_values[last_voiced]))
            
    return turns

### Extract Conversation Dynamics

Turn length.

In [ ]:
def get_overlapping_turns(turn, turns):
    """
    Get the (partly) overlapping turns of a turn.

    Args:
    - turn (start_time, end_time): The turn of interest.
    - turns2 (list of tuples): [(start_time, end_time), ...] representing the turns to compare.

    Returns:
    - list of tuples: [(start_time, end_time), ...] representing the (partly) overlapping turns.
    """
    # Condition 1: Turn of interest starts within another turn in turns
    condition1 = [t for t in turns if t[0] <= turn[0] <= t[1]]

    # Condition 2: Turn of interest ends within another turn in turns
    condition2 = [t for t in turns if t[0] <= turn[1] <= t[1]]

    # Condition 3: Turn of interest starts before another turn and ends after another turn
    condition3 = [t for t in turns if turn[0] <= t[0] <= t[1] <= turn[1]]

    # Create a union of three lists (turns that satisfy any of the conditions)
    union = condition1 + condition2 + condition3

    # Remove duplicates
    return list(set(union))

In [ ]:
def get_turn_lengths(turns1, turns2):
    """
    Get the turn length of the first participant (i.e., the durations of uninterrupted speech).

    Args:
    - turns1 (list of tuples): [(start_time, end_time), ...] representing the turns of the first participant.
    - turns2 (list of tuples): [(start_time, end_time), ...] representing the turns of the second participant.

    Returns:
    - list: The turn lengths for the first participant.
    """
    turn_lengths = [] # List to store the turn lengths

    # Iterate through the turns of the first participant
    for t1 in range(len(turns1)):
        start = turns1[t1][0] # Start of the turn
        end = turns1[t1][1] # End of the turn

        # Calculate the total length of turn t1
        total_length = end - start

        # Find turns in turns2 that overlap with turn t1
        overlapping_turns = get_overlapping_turns(turns1[t1], turns2)

        # Substract overlapping turn lengths
        for t2 in overlapping_turns:
            if start <= t2[0] and end >= t2[1]: # Completely overlapping
                total_length -= t2[1] - t2[0]
            elif t2[0] <= start: # Overlapping at the beginning
                total_length -= t2[1] - start
            else: # Overlapping at the end
                total_length -= end - t2[0]

        turn_lengths.append(max(total_length, 0)) # Ensure non-negative length

    return turn_lengths

Speaking time.

In [ ]:
def get_speaking_time(turn_lengths, interaction_duration):
    """
    Calculate the percentage of total speaking time of the speaker of interest.

    Args:
    - turn_lengths (np.array): The turn lengths of the speaker of interest.
    - interaction_duration (int): The total interaction duration in seconds.

    Returns:
    - float: The percentage of total speaking time for the speaker of interest.
    """
    return (np.sum(turn_lengths) / interaction_duration) * 100

Response time.

In [ ]:
def get_response_times(turns1, turns2):
    """
    Calculate the response times of the first speaker.

    Args:
    - turns1 (list of tuples): [(start_time, end_time), ...] representing turns of the first speaker.
    - turns2 (list of tuples): [(start_time, end_time), ...] representing turns of the second speaker.

    Returns:
    - list: The response times of the first speaker.
    """
    response_times = []

    for i, (start, end) in enumerate(turns1):
        # Find the last preceding turn of the second speaker
        last_turn2 = next((t for t in reversed(turns2) if t[0] <= start), None)

        if last_turn2 is None: # No turn found
            response_times.append(0) 
        else: # Turn found
            # Calculate the response time of the first speaker, set to 0 if speech overlaps
            response_times.append(max(start - last_turn2[1], 0))
            
    return response_times

Speech rate.

In [ ]:
def transcribe_turn(sound, start_time, end_time, model):
    """
    Transcribe the turn segment of the audio sound.

    Args:
    - sound (parselmouth.Sound): The audio sound.
    - start_time (float): The start time of the turn segment.
    - end_time (float): The end time of the turn segment.
    - model (whisper.Model): The preloaded Whisper model for transcription.

    Returns:
    - str: The transcription of the turn segment.
    """
    # Extract the turn segment of the audio
    segment = sound.extract_part(from_time=start_time, to_time=end_time)

    # Save the segment to a temporary WAV file
    segment_wav_path = "temp_segment.wav"
    segment.save(segment_wav_path, "WAV")

    # Transcribe the audio segment
    result = model.transcribe(segment_wav_path)

    # Get the transcription text
    transcription = result["text"]

    # Delete the temporary WAV file
    os.remove(segment_wav_path)

    return transcription

In [ ]:
def transcription_file_name(global_pid, batch_id, dyad_id, setting, start_time, end_time):
    """
    Generate the file name for the transcription based on the parameters.

    Args:
    - global_pid (int): The global participant ID.
    - batch_id (str): The batchID value of the turn (e.g. "Batch_8").
    - dyad_id (str): The dyadID value of the turn.
    - setting (str): The setting of the recording file (i.e., "conv" or "coll").
    - start_time (float): The start time of the turn segment.
    - end_time (float): The end time of the turn segment.

    Returns:
    - str: The file name for the transcription.
    """
    if setting == "conv":
        return f"data/Conversation/Transcriptions/{batch_id}/{dyad_id}_{global_pid}_{start_time:.2f}_{end_time:.2f}"
    elif setting == "coll":
        return f"data/Collaboration/Transcriptions/{batch_id}/{dyad_id}_{global_pid}_{start_time:.2f}_{end_time:.2f}"
    else:   
        raise ValueError("Invalid setting. Please specify 'conv' or 'coll'.") 

In [ ]:
def store_transcription(transcription, file_name):
    """
    Store the transcription in a text file.

    Args:
    - transcription (str): The transcription text.
    - file_name (str): The file name for the transcription.

    Returns: None
    """
    # Create the directory if it doesn't exist
    os.makedirs(os.path.dirname(file_name), exist_ok=True)

    # Write the transcription to the text file
    with open(f"{file_name}.txt", "w") as f:
        f.write(transcription)

In [ ]:
def get_transcription(global_pid, batch_id, dyad_id, setting, sound, start_time, end_time, model):
    """
    Get the transcription of the turn segment.
    - If the transcription file already exists, read the transcription from the file.
    - If the transcription file does not exist, transcribe the turn segment and store it in a text file.

    Args:
    - global_pid (int): The globalPID of the participant.
    - batch_id (str): The batchID value of the turn (e.g. "Batch_8").
    - dyad_id (str): The dyadID value of the turn.
    - setting (str): The setting of the turn (i.e., "conv" or "coll").
    - sound (parselmouth.Sound): The audio sound.
    - start_time (float): The start time of the turn segment.
    - end_time (float): The end time of the turn segment.
    - model (whisper.Model): The preloaded Whisper model for transcription.

    Returns:
    - str: The transcription of the turn segment.
    """
    # Generate the transcription file name
    file_name = transcription_file_name(global_pid, batch_id, dyad_id, setting, start_time, end_time)

    # Check if the transcription file already exists
    if os.path.isfile(f"{file_name}.txt"):
        # Extract transcription
        with open(f"{file_name}.txt", "r") as f:
            transcription = f.read()
    else:
        # Transcribe the turn segment and store it in a text file
        transcription = transcribe_turn(sound, start_time, end_time, model)
        store_transcription(transcription, file_name)
    
    return transcription
    

In [ ]:
def calculate_wpm(transcription, duration_minutes):
    """
    Calculate the words per minute (WPM) of the transcription.

    Args:
    - transcription (str): The transcription of the turn segment.
    - duration_minutes (float): The duration of the turn segment in minutes.

    Returns:
    - float: The words per minute (WPM) of the turn.
    """
    # Count the number of words in the transcription
    words = re.findall(r'\b\w+\b', transcription)
    num_words = len(words)

    # Calculate the words per minute (WPM)
    wpm = num_words / duration_minutes if duration_minutes > 0 else 0

    return wpm

In [ ]:
def get_speech_rates(global_pid, batch_id, dyad_id, setting, turns, sound, model):
    """
    Calculate the speech rate (words per minute) for each turn.

    Args:
    - global_pid (int): The globalPID of the participant.
    - batch_id (str): The batchID value of the turn (e.g. "Batch_8").
    - dyad_id (str): The dyadID value of the turn.
    - setting (str): The setting of the turn (i.e., "conv" or "coll").
    - turns (list of tuples): [(start_time, end_time), ...] representing turns.
    - sound (parselmouth.Sound): The audio sound.
    - model (whisper.Model): The preloaded Whisper model for transcription.

    Returns:
    - list: The speech rates for each turn.
    """
    speech_rates = []

    # Calculate words per minute in each turn
    for start_time, end_time in turns:
        # Get the transcription of the turn
        transcription = get_transcription(global_pid, batch_id, dyad_id, setting, sound, start_time, end_time, model)

        # Calculate the duration of the turn in minutes
        duration_minutes = (end_time - start_time) / 60

        # Calculate the words per minute (WPM) of the turn
        wpm = calculate_wpm(transcription, duration_minutes)
        speech_rates.append(wpm)

    return speech_rates

Extract the four conversation dynamics.

In [ ]:
def conversation_dynamics(self_recording_file, other_recording_file, setting, batch_id, dyad_id, self_global_pid, other_global_pid, model):
    """
    Extract the conversation dynamics from the recording files of the self and other participant.

    Args:
    - self_recording_file (str): The name of the self participant recording file.
    - other_recording_file (str): The name of the other participant recording file.
    - setting (str): The setting of the recording files (i.e., "conv" or "coll").
    - batch_id (str): The batchID value of the recording files (e.g. "Batch_8").
    - dyad_id (str): The dyadID value of the recording files.
    - self_global_pid (int): The globalPID of the self participant.
    - other_global_pid (int): The globalPID of the other participant.
    - model (whisper.Model): The preloaded Whisper model for transcription (speech rate).
    
    Returns:
    - list of tuples: [(start_time, end_time), ...] representing the turns of the self participant.
    - list: The conversation dynamics of the self participant.
    - list of tuples: [(start_time, end_time), ...] representing the turns of the other participant.
    - list: The conversation dynamics of the other participant.
    """
    # Extract the sounds from the recording files
    self_snd = extract_sound(self_recording_file, setting, batch_id)
    other_snd = extract_sound(other_recording_file, setting, batch_id)

    # Trim the sounds to 3 minutes
    self_snd, self_snd_duration = trim(self_snd)
    other_snd, other_snd_duration = trim(other_snd)

    # Extract the time and intensity values
    self_time_values, self_intensity_values = extract_time_intensity(self_snd)
    other_time_values, other_intensity_values = extract_time_intensity(other_snd)

    # Extract the turns
    self_turns = get_turns(self_time_values, self_intensity_values)
    other_turns = get_turns(other_time_values, other_intensity_values)

    # Extract the turn lengths
    self_turn_lengths = get_turn_lengths(self_turns, other_turns)
    other_turn_lengths = get_turn_lengths(other_turns, self_turns)

    # Extract the speaking time
    self_speaking_time = get_speaking_time(self_turn_lengths, self_snd_duration)
    other_speaking_time = get_speaking_time(other_turn_lengths, other_snd_duration)
    
    # Extract the speech rates
    self_speaking_rate = get_speech_rates(self_global_pid, batch_id, dyad_id, setting, self_turns, self_snd, model)
    other_speaking_rate = get_speech_rates(other_global_pid, batch_id, dyad_id, setting, other_turns, other_snd, model)

    # Extract the response times
    self_response_times = get_response_times(self_turns, other_turns)
    other_response_times = get_response_times(other_turns, self_turns)

    return self_turns, [self_speaking_time, self_turn_lengths, self_speaking_rate, self_response_times], other_turns, [other_speaking_time, other_turn_lengths, other_speaking_rate, other_response_times]

### Compute Measures of Conversation Dynamics

In [ ]:
def align_time_series(self_turns, self_conversation_dynamic, other_turns, other_conversation_dynamic):
    """
    Align the conversation dynamic timelines (self speaker t <-> other speaker t - 1).
    If one participant had consecutive turns without the other participant intervening, 
    we averaged the conversation dynamics of those consecutive turns.

    Args:
    - self_turns (list of tuples): [(start_time, end_time), ...] representing the turns of the self participant.
    - self_conversation_dynamic (list): The conversation dynamics of the self participant.
    - other_turns (list of tuples): [(start_time, end_time), ...] representing the turns of the other participant.
    - other_conversation_dynamic (list): The conversation dynamics of the other participant.

    Returns:
    - np.array: The aligned conversation dynamic values for the self participant.
    - np.array: The aligned conversation dynamic values for the other participant.
    """
    # Create empty lists to store the aligned conversation dynamics
    aligned_self_conversation_dynamics = []
    aligned_other_conversation_dynamics = []

    other_t = 0 # First turn of the other participant

    # First turn of the self participant following a turn of the other participant
    self_t = next((i for i, t in enumerate(self_turns) if t[0] > other_turns[other_t][0]), len(self_turns))

    # Keep track of the current speaker turn
    current = "other" # "self" or "other"
    consecutive_turns = 0 # Number of consecutive turns
    current_dynamic_sum = 0 # Sum of the conversation dynamics for the current speaker turn

    # Iterate through the turns of both participants
    while self_t < len(self_turns) and other_t < len(other_turns):
        # Find the current turn for both participants
        self_turn = self_turns[self_t][0]
        other_turn = other_turns[other_t][0]

        # Check if the current speaker is the other participant
        if current == "other":
            if self_turn <= other_turn: # End aligned turn
                # Calculate the average conversation dynamic
                aligned_other_conversation_dynamics.append(current_dynamic_sum / consecutive_turns)

                # Update the current speaker to the self participant
                current = "self"
                consecutive_turns = 0
                current_dynamic_sum = 0

            else: # Continue aligned turn
                consecutive_turns += 1 # Increase the number of consecutive turns
                current_dynamic_sum += other_conversation_dynamic[other_t] # Add the current conversation dynamic

                # Move to the next turn of the other participant
                other_t += 1

        else: # Check if the current speaker is the self participant
            if self_turn <= other_turn: # Continue aligned turn
                consecutive_turns += 1 # Increase the number of consecutive turns
                current_dynamic_sum += self_conversation_dynamic[self_t] # Add the current conversation dynamic

                # Move to the next turn of the self participant
                self_t += 1

            else: # End aligned turn
                # Calculate the average conversation dynamic
                aligned_self_conversation_dynamics.append(current_dynamic_sum / consecutive_turns)

                # Update the current speaker to the other participant
                current = "other"
                consecutive_turns = 0
                current_dynamic_sum = 0

    # Handle the last turn
    if current == "self": # Last turn was from the self participant
        aligned_self_conversation_dynamics.append(current_dynamic_sum / consecutive_turns)
    else: # Last turn was from the other participant
        aligned_other_conversation_dynamics.append(current_dynamic_sum / consecutive_turns)

        # Combine the remaining turns of the self participant
        consecutive_turns = 0
        current_dynamic_sum = 0

        for i in range(self_t, len(self_turns)):
            consecutive_turns += 1
            current_dynamic_sum += self_conversation_dynamic[i]

        aligned_self_conversation_dynamics.append(current_dynamic_sum / consecutive_turns)

    # Remove the last element of the other participant's conversation dynamics if it is longer than the self participant's
    # This is done to ensure that conversation dynamics lists have the same length
    if len(aligned_other_conversation_dynamics) > len(aligned_self_conversation_dynamics):
        aligned_other_conversation_dynamics = aligned_other_conversation_dynamics[:-1] # Remove the last element

    return aligned_self_conversation_dynamics, aligned_other_conversation_dynamics

In [ ]:
def get_measures(self_turns, self_conversation_dynamic, other_turns, other_conversation_dynamic):
    """
    Calculate the median, coefficient of variation, adaptability and predictability of 
    the values of a specific conversation dynamic of the self participant.

    Args:
    - self_turns (list of tuples): [(start_time, end_time), ...] representing the turns of the self participant.
    - self_conversation_dynamic (list): The values of a specific conversation dynamic of the self participant.
    - other_turns (list of tuples): [(start_time, end_time), ...] representing the turns of the other participant.
    - other_conversation_dynamic (list): The values of a specific conversation dynamic of the other participant.

    Returns:
    - float: The median value of the conversation dynamic.
    - float: The coefficient of variation of the conversation dynamic.
    - float: The adaptability of the conversation dynamic.
    - float: The predictability of the conversation dynamic.
    """
    # Median
    median = np.median(self_conversation_dynamic)

    # Coefficient of variation (i.e., ratio between the standard deviation and the mean)
    if np.mean(self_conversation_dynamic) > 0:
        variation = np.std(self_conversation_dynamic) / np.mean(self_conversation_dynamic)
    else:
        variation = 0

    self_aligned_conversation_dynamic, other_aligned_conversation_dynamic = align_time_series(self_turns, self_conversation_dynamic, other_turns, other_conversation_dynamic)

    # Adaptability (i.e., Spearman correlation between conversation dynamic values of the first at time t and second speaker at time t - 1)
    if len(self_conversation_dynamic) > 1:
        adaptability = np.corrcoef(self_aligned_conversation_dynamic, other_aligned_conversation_dynamic)[0, 1]

    # Predictability (i.e., Spearman correlation between conversation dynamic values at time t and t - 1)
    if len(self_conversation_dynamic) > 1:
        predictability = np.corrcoef(self_conversation_dynamic[1:], self_conversation_dynamic[:-1])[0, 1]

    return median, variation, adaptability, predictability

In [ ]:
def store_measures(df, setting, names_conversation_dynamics, names_measures, model):
    """
    Store the conversation dynamics measures in the dataframe.

    Args:
    - df (pd.DataFrame): The dataframe containing the questionnaire data.
    - setting (str): The setting of the questionnaire data (i.e., "conv" or "coll").
    - names_conversation_dynamics (list): The names of the conversation dynamics.
    - names_measures (list): The names of the measures.
    - model (whisper.Model): The preloaded Whisper model for transcription (speech rate).

    Returns:
    - pd.DataFrame: The dataframe with the conversation dynamics measures.
    """
    # Create a new dataframe to store the conversation dynamics measures
    df_measures = df.copy()

    # Iterate over the dataframe
    # for interaction in df_measures.itertuples():
    for interaction in tqdm(df_measures.itertuples(), total=len(df_measures), desc=f"Processing {setting} interactions"):
        # Extract the conversation dynamics for each dyad
        self_turns, self_conversation_dynamics, other_turns, other_conversation_dynamics = conversation_dynamics(
            interaction.self_recording_file,
            interaction.other_recording_file,
            setting,
            interaction.batchID,
            interaction.dyadID,
            interaction.selfPID,
            interaction.otherPID,
            model
    )

        # Store speaking time in the dataframe
        df_measures.loc[interaction.Index, "self_speaking_time"] = self_conversation_dynamics[0]
        df_measures.loc[interaction.Index, "other_speaking_time"] = other_conversation_dynamics[0]

        for c in range(1, len(self_conversation_dynamics)): # Skip the first element (i.e., speaking time)
            # Calculate the measures for the self participant
            self_measures = get_measures(self_turns, self_conversation_dynamics[c], other_turns, other_conversation_dynamics[c])

            # Store the measures in the dataframe
            for m in range(len(self_measures)):
                df_measures.loc[interaction.Index, f"self_{names_conversation_dynamics[c]}_{names_measures[m]}"] = self_measures[m]

            # Calculate the measures for the other participant
            other_measures = get_measures(other_turns, other_conversation_dynamics[c], self_turns, self_conversation_dynamics[c])

            # Store the measures in the dataframe
            for m in range(len(other_measures)):
                df_measures.loc[interaction.Index, f"other_{names_conversation_dynamics[c]}_{names_measures[m]}"] = other_measures[m]

    return df_measures


### Apply Computation to the Dataframes

In [ ]:
# Load the Whisper model for transcription
model = whisper.load_model("base.en")

In [ ]:
# Names of the conversation dynamics and measures
names_conversation_dynamics = ["speaking_time", "turn_lengths", "speaking_rate", "response_times"]
names_measures = ["median", "variability", "adaptability", "predictability"]

Compute and store measures.

In [ ]:
# Suppress a specific warning (that occurs often but is not important for us)
warnings.filterwarnings("ignore", message="FP16 is not supported on CPU; using FP32 instead")

df_measures_conv = store_measures(final_df_combined_conv, "conv", names_conversation_dynamics, names_measures, model)
df_measures_coll = store_measures(final_df_combined_coll, "coll", names_conversation_dynamics, names_measures, model)

## Transform Data

### Split Rows

In [ ]:
def split_rows(df, setting):
    """
    Split the dyad rows into two rows: one for self participant and one for other participant.
    In other words, this would result in two times more rows.
    
    For the self participant, the self_ and other_ columns are kept as is.
    For the other participant, the self_ and other_ columns are swapped.
    
    Perceived rapport values are put into the column 'rapport'.
    The original self_{setting}_CC_rapp1 and other_{setting}_CC_rapp1 columns are dropped.

    Args:
    - df (pd.DataFrame): The dataframe containing the questionnaire data.
    - setting (str): The setting of the questionnaire data (i.e., "conv" or "coll").

    Returns:
    - pd.DataFrame: The transformed dataframe.
    """
    # Create a copy of the dataframe
    df_transformed = df.copy()

    # Create a new column 'rapport' and assign values from 'self_{setting}_CC_rapp1'
    df_transformed['rapport'] = df_transformed[f'self_{setting}_CC_rapp1']

    # Create rows for 'other_{setting}_CC_rapp1' with swapped values
    other_rows = df_transformed.copy()
    other_rows['rapport'] = df_transformed[f'other_{setting}_CC_rapp1']

    # Swap self_ and other_ columns for the 'other' rows
    for col in df_transformed.columns:
        if col.startswith('self'):
            other_rows[col] = df_transformed[col.replace('self', 'other')]
        elif col.startswith('other'):
            other_rows[col] = df_transformed[col.replace('other', 'self')]

    # Concatenate the original and transformed ('other') rows
    df_combined = pd.concat([df_transformed, other_rows], ignore_index=True)

    # Drop the original self_{setting}_CC_rapp1 and other_{setting}_CC_rapp1 columns
    df_combined = df_combined.drop(columns=[f'self_{setting}_CC_rapp1', f'other_{setting}_CC_rapp1'])

    return df_combined

In [ ]:
# Split the dyad rows of the dataframes into participant-specific rows
df_split_measures_conv = split_rows(df_measures_conv, "conv")
df_split_measures_coll = split_rows(df_measures_coll, "coll")

print("Number of rows in the original dataframes:", df_measures_conv.shape[0], "and", df_measures_coll.shape[0])
print("Number of rows in the split dataframes:", df_split_measures_conv.shape[0], "and", df_split_measures_coll.shape[0])

### Combine Datasets

In [ ]:
# Create copies of the dataframes
df_split_measures_conv_copy = df_split_measures_conv.copy()
df_split_measures_coll_copy = df_split_measures_coll.copy()

# Combine the dataframes by introducing a new column 'setting'
df_split_measures_conv_copy["setting"] = 0
df_split_measures_coll_copy["setting"] = 1

# Concatenate the dataframes
df_split_measures_combined = pd.concat([df_split_measures_conv_copy, df_split_measures_coll_copy], ignore_index=True)
df_split_measures_combined

### Standardization

In [ ]:
def standardize(df):
    """
    Standardize the numeric columns (except perceived rapport and setting) in the dataframe.
    This function groups columns with similar base names, indicating measures (e.g. speaking_time),
    and standardizes them together.

    Args:
    - df (pd.DataFrame): The dataframe containing the data.

    Returns:
    - pd.DataFrame: The dataframe with standardized columns.
    """
    # Create a copy of the dataframe to avoid modifying the original
    df_copy = df.copy()

    # All numeric columns
    columns = df_copy.select_dtypes(include=[np.number]).columns

    # Exclude perceived rapport and setting columns
    exclude_columns = ["rapport", "setting"]
    columns = [col for col in columns if col not in exclude_columns]
    print('Standardized columns:', columns)
    print('Non-standardized columns:', list(df_copy.columns.difference(columns)))

    # Group by base name (e.g. speaking_time, turn_lengths, etc.)
    grouped = {}
    for col in columns:
        match = re.match(r'(self|other)_(.+)', col)
        if match:
            base = match.group(2)
            grouped.setdefault(base, []).append(col)
        else:
            # Handle columns without self_/other_ prefix
            grouped.setdefault(col, []).append(col)

    print('Groups:', grouped)

    # Standardize each group together
    for base, cols in grouped.items():
        if len(cols) > 1:
            values = df_copy[cols].values
            scaler = StandardScaler()
            df_copy[cols] = scaler.fit_transform(values)
        else:
            # Standardize solo columns normally
            scaler = StandardScaler()
            df_copy[cols] = scaler.fit_transform(df_copy[cols])

    return df_copy

In [ ]:
df_stnd_split_measures_conv = standardize(df_split_measures_conv)
df_stnd_split_measures_coll = standardize(df_split_measures_coll)
df_stnd_split_measures_combined = standardize(df_split_measures_combined)

### Interaction Terms

In [ ]:
# Create interaction terms for the self_ and other_ columns with setting
for col in df_stnd_split_measures_combined.columns:
    if (col.startswith('self_') or col.startswith('other_')) and not col.endswith("recording_file"):
        df_stnd_split_measures_combined[f"setting_x_{col}"] = df_stnd_split_measures_combined["setting"] * df_stnd_split_measures_combined[col]

print(df_stnd_split_measures_combined.columns)

## Data Analysis

Dataframe:
- Person ID (globalPID)
- Gender (....)
- Batch (batchID)
- Group size (...)
- Setting (Conversation / Collaboration)
- Rapport
- Conversation dynamics (self)
    - Speaking time
        - Percentage
    - Turn length / Response time / Speech rate:
        - Median
        - Variation
        - Adaptability
        - Predictability
- Conversation dynamics (other)
    - Speaking time
        - Percentage
    - Turn length / Response time / Speech rate:
        - Median
        - Variation
        - Adaptability
        - Predictability
<!-- - Participant 1 (PID)
- Gender participant 1 
- Participant 2 (PID)
- Gender participant 2
- Batch (batchID)
- Group size
- Setting (Conversation / Collaboration)
- Rapport participant 1
- Conversation dynamics participant 1
    - Speaking time
        - Percentage
    - Turn length / Response time / Speech rate:
        - Median
        - Variation
        - Adaptability
        - Predictability
- Conversation dynamics participant 2
    - Speaking time
        - Percentage
    - Turn length / Response time / Speech rate:
        - Median
        - Variation
        - Adaptability
        - Predictability -->

### Help Functions

In [ ]:
def get_correlation_matrix(df, setting, include_self, include_other, include_interactions, annotate=False, name_mapping=None):
    """
    Get the correlation matrix of the conversation dynamics measures.

    Args:
    - df (pd.DataFrame): The dataframe containing the conversation dynamics measures.
    - setting (str): The setting of the conversation dynamics measures (i.e., "conv", "coll" or "both").
    - include_self (bool): Whether to include the self participant measures in the correlation matrix.
    - include_other (bool): Whether to include the other participant measures in the correlation matrix.
    - include_interactions (bool): Whether to include the interaction terms in the correlation matrix.
    - annotate (bool): Whether to annotate the correlation matrix with the correlation values.
    - name_mapping (dict): A dictionary mapping the original technical column names to the new nice readable names.

    Returns:
    - pd.DataFrame: The correlation matrix of the conversation dynamics measures.
    """
    # Create a list of conversation dynamics measures
    conv_dyn_measures = []

    # Include the conversation dynamics measures for the self participant
    if include_self:
        conv_dyn_measures += ["self_speaking_time"]
        conv_dyn_measures += [f"self_{c}_{m}" for c in names_conversation_dynamics[1:] for m in names_measures]
    
    # Include the conversation dynamics measures for the other participant
    if include_other:
        conv_dyn_measures += ["other_speaking_time"]
        conv_dyn_measures += [f"other_{c}_{m}" for c in names_conversation_dynamics[1:] for m in names_measures]

    # Include the interaction terms
    if include_interactions:
        conv_dyn_measures += ["setting"]
        if include_self:
            conv_dyn_measures += [f"setting_x_self_speaking_time"]
            conv_dyn_measures += [f"setting_x_self_{c}_{m}" for c in names_conversation_dynamics[1:] for m in names_measures]
        if include_other:
            conv_dyn_measures += [f"setting_x_other_speaking_time"]
            conv_dyn_measures += [f"setting_x_other_{c}_{m}" for c in names_conversation_dynamics[1:] for m in names_measures]

    # Compute the correlation matrix
    correlation_matrix = df[conv_dyn_measures].corr()
    
    # Rename rows and columns to use nice readable names
    if name_mapping is not None:
        correlation_matrix.rename(columns=name_mapping, index=name_mapping, inplace=True)

    # Generate the title of the correlation matrix
    if setting == "conv":
        title = "Conversation Dynamics Measures - Conversation Setting"

    elif setting == "coll":
        title = "Conversation Dynamics Measures - Collaboration Setting"

    elif setting == "both":
        title = "Conversation Dynamics Measures - Both Settings"

    # Plot the correlation matrix
    plt.figure(figsize=(12, 8))
    sns.heatmap(correlation_matrix, annot=annotate, fmt=".2f", cmap="coolwarm", square=True, cbar_kws={"shrink": .8})
    plt.title(title)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

    return correlation_matrix

In [ ]:
def highly_correlated_pairs(correlation_matrix, threshold=0.7):
    """
    Get highly correlated pairs of variables from the correlation matrix.

    Args:
    - correlation_matrix (pd.DataFrame): The correlation matrix.
    - threshold (float): The threshold for high correlation (default is 0.7).

    Returns:
    - list: A list of tuples representing the highly correlated pairs.
    """
    # Get the upper triangle of the correlation matrix
    upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))

    # Find pairs with absolute correlation greater than the threshold
    highly_correlated = [(col1, col2) for col1 in upper_triangle.columns for col2 in upper_triangle.index if abs(upper_triangle[col1][col2]) > threshold]

    # Print the highly correlated pairs
    print("Highly correlated pairs:", (len(highly_correlated)))
    for (p1, p2) in highly_correlated:
        print(f"{p1} and {p2}: {correlation_matrix.loc[p1, p2]:.2f}")

    return highly_correlated

In [ ]:
def calculate_vif(df, include_prefixes=None, include_setting=False, exclude_columns=None):
    """
    Calculate VIF for selected columns in a dataframe.
    
    Parameters:
    - df (pd.DataFrame): The input dataframe
    - include_prefixes (tuple or list of str): Column name prefixes to include (e.g., ("self_", "other_"))
    - include_setting (bool): Whether to include the 'setting' column in the analysis
    - exclude_columns (list of str): Column names to exclude from the analysis

    Returns: None
    """
    # Filter included columns (based on prefix)
    if include_prefixes is not None:
        included_cols = [col for col in df.columns if col.startswith(tuple(include_prefixes))]
    else:
        included_cols = list(df.columns)

    # Include the 'setting' column if specified
    if include_setting:
        included_cols.append("setting")
    
    # Remove excluded columns
    if exclude_columns is not None:
        predictor_vars = [col for col in included_cols if col not in exclude_columns]
    else:
        predictor_vars = included_cols

    # Add constant
    X = add_constant(df[predictor_vars])

    # Calculate VIF
    vif_data = pd.DataFrame({
        "Variable": X.columns,
        "VIF": [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    })

    print(vif_data)

In [ ]:
# Mapping from technical names to nice readable names (for plotting)
name_mapping = {
    "self_speaking_time": "Self speaking time",
    "other_speaking_time": "Other speaking time",
    "self_turn_lengths_median": "Self turn length (median)",
    "self_turn_lengths_variability": "Self turn length (variability)",
    "self_turn_lengths_adaptability": "Self turn length (adaptability)",
    "self_turn_lengths_predictability": "Self turn length (predictability)",
    "other_turn_lengths_median": "Other turn length (median)",
    "other_turn_lengths_variability": "Other turn length (variability)",
    "other_turn_lengths_adaptability": "Other turn length (adaptability)",
    "other_turn_lengths_predictability": "Other turn length (predictability)",
    "self_speaking_rate_median": "Self speech rate (median)",
    "self_speaking_rate_variability": "Self speech rate (variability)",
    "self_speaking_rate_adaptability": "Self speech rate (adaptability)",
    "self_speaking_rate_predictability": "Self speech rate (predictability)",
    "other_speaking_rate_median": "Other speech rate (median)",
    "other_speaking_rate_variability": "Other speech rate (variability)",
    "other_speaking_rate_adaptability": "Other speech rate (adaptability)",
    "other_speaking_rate_predictability": "Other speech rate (predictability)",
    "self_response_times_median": "Self response time (median)",
    "self_response_times_variability": "Self response time (variability)",
    "self_response_times_adaptability": "Self response time (adaptability)",
    "self_response_times_predictability": "Self response time (predictability)",
    "other_response_times_median": "Other response time (median)",
    "other_response_times_variability": "Other response time (variability)",
    "other_response_times_adaptability": "Other response time (adaptability)",
    "other_response_times_predictability": "Other response time (predictability)",
    "setting": "Setting",

    # Interaction terms
    "setting_x_self_speaking_time": "Setting × Self speaking time",
    "setting_x_other_speaking_time": "Setting × Other speaking time",
    "setting_x_self_turn_lengths_median": "Setting × Self turn length (median)",
    "setting_x_self_turn_lengths_variability": "Setting × Self turn length (variability)",
    "setting_x_self_turn_lengths_adaptability": "Setting × Self turn length (adaptability)",
    "setting_x_self_turn_lengths_predictability": "Setting × Self turn length (predictability)",
    "setting_x_other_turn_lengths_median": "Setting × Other turn length (median)",
    "setting_x_other_turn_lengths_variability": "Setting × Other turn length (variability)",
    "setting_x_other_turn_lengths_adaptability": "Setting × Other turn length (adaptability)",
    "setting_x_other_turn_lengths_predictability": "Setting × Other turn length (predictability)",
    "setting_x_self_speaking_rate_median": "Setting × Self speech rate (median)",
    "setting_x_self_speaking_rate_variability": "Setting × Self speech rate (variability)",
    "setting_x_self_speaking_rate_adaptability": "Setting × Self speech rate (adaptability)",
    "setting_x_self_speaking_rate_predictability": "Setting × Self speech rate (predictability)",
    "setting_x_other_speaking_rate_median": "Setting × Other speech rate (median)",
    "setting_x_other_speaking_rate_variability": "Setting × Other speech rate (variability)",
    "setting_x_other_speaking_rate_adaptability": "Setting × Other speech rate (adaptability)",
    "setting_x_other_speaking_rate_predictability": "Setting × Other speech rate (predictability)",
    "setting_x_self_response_times_median": "Setting × Self response time (median)",
    "setting_x_self_response_times_variability": "Setting × Self response time (variability)",
    "setting_x_self_response_times_adaptability": "Setting × Self response time (adaptability)",
    "setting_x_self_response_times_predictability": "Setting × Self response time (predictability)",
    "setting_x_other_response_times_median": "Setting × Other response time (median)",
    "setting_x_other_response_times_variability": "Setting × Other response time (variability)",
    "setting_x_other_response_times_adaptability": "Setting × Other response time (adaptability)",
    "setting_x_other_response_times_predictability": "Setting × Other response time (predictability)",

    # Robustness check
    "self_age": "Self age",
    "other_age": "Other age",
    "self_sex": "Self sex",
    "other_sex": "Other sex",
}

### Conversation Setting (RQ1)

#### Correlations

Measures of one participant.

In [ ]:
correlation_matrix_self_conv = get_correlation_matrix(df_stnd_split_measures_conv, "conv", True, False, False, False, name_mapping)
high_corr_pairs_self_conv = highly_correlated_pairs(correlation_matrix_self_conv)

Measures of both participants.

In [ ]:
correlation_matrix_both_conv = get_correlation_matrix(df_stnd_split_measures_conv, "conv", True, True, False, False, name_mapping)
high_corr_pairs_both_conv = highly_correlated_pairs(correlation_matrix_both_conv)

#### Variation Inflation Factor (VIF)

Measures of both participants.

In [ ]:
# Calculate VIF for self_ and other_ measures
calculate_vif(df_stnd_split_measures_conv, include_prefixes=["self_", "other_"], exclude_columns=["self_recording_file", "other_recording_file"])

#### Remove Multicollinearity

Correlations between self_speaking_time/other_speaking_time and rapport.

In [ ]:
# Correlation between rapport and self_speaking_time and other_speaking_time
corr_rapport_self = df_stnd_split_measures_conv["self_speaking_time"].corr(df_stnd_split_measures_conv["rapport"])
corr_rapport_other = df_stnd_split_measures_conv["other_speaking_time"].corr(df_stnd_split_measures_conv["rapport"])

print(f"Correlation between self_speaking_time and rapport: {corr_rapport_self:.2f}")
print(f"Correlation between other_speaking_time and rapport: {corr_rapport_other:.2f}")

VIF after excluding self_speaking_time.

In [ ]:
calculate_vif(df_stnd_split_measures_conv, include_prefixes=["self_", "other_"], exclude_columns=["self_recording_file", "other_recording_file", "self_speaking_time"])

#### Cumulative Link-Mixed Model (CLMM)

Formula.

In [ ]:
# All conversation dynamics measures (except self speaking time)
formula_full_other = (
    'rapport ~ other_speaking_time + '
    'self_turn_lengths_median + self_turn_lengths_variability + '
    'self_turn_lengths_adaptability + self_turn_lengths_predictability + '
    'other_turn_lengths_median + other_turn_lengths_variability + '
    'other_turn_lengths_adaptability + other_turn_lengths_predictability + '
    'self_speaking_rate_median + self_speaking_rate_variability + '
    'self_speaking_rate_adaptability + self_speaking_rate_predictability + '
    'other_speaking_rate_median + other_speaking_rate_variability + '
    'other_speaking_rate_adaptability + other_speaking_rate_predictability + '
    'self_response_times_median + self_response_times_variability + '
    'self_response_times_adaptability + self_response_times_predictability + '
    'other_response_times_median + other_response_times_variability + '
    'other_response_times_adaptability + other_response_times_predictability + '
    '(1 | dyadID) + (1 | selfPID)'
)

ro.globalenv['formula_full_other'] = formula_full_other

No weights.

In [ ]:
# Enable conversion between Pandas and R data frames
pandas2ri.activate()

# Load R's 'ordinal' package
ordinal = importr("ordinal")

# Convert to R dataframe
r_df_conv = pandas2ri.py2rpy(df_stnd_split_measures_conv)

# Define the CLMM model in R
ro.r.assign("r_df_conv", r_df_conv)
ro.r('''
# install.packages("ordinal") # Uncomment if you need to install the package
library(ordinal)
     
r_df_conv$rapport <- as.ordered(r_df_conv$rapport)  # Ensure it's ordinal

# Fit the model
clmm_model_full_other <- clmm(as.formula(formula_full_other), data = r_df_conv)
     
# Save the model summary to a text file     
sink("models/RQ1/no_weights/clmm_full_other.txt") # Redirect output to a file
print(summary(clmm_model_full_other))
sink() # Stop redirecting output
     
# Save the model to a file
saveRDS(clmm_model_full_other, file = "models/RQ1/comparison/clmm_full_other_unweighted.rds")
''')

Weights.

In [ ]:
# Enable conversion between Pandas and R data frames
pandas2ri.activate()

# Load R's 'ordinal' package
ordinal = importr("ordinal")

# Convert to R dataframe
r_df_conv = pandas2ri.py2rpy(df_stnd_split_measures_conv)

# Define the CLMM model in R
ro.r.assign("r_df_conv", r_df_conv)
ro.r('''
# install.packages("ordinal") # Uncomment if you need to install the package
library(ordinal)
     
r_df_conv$rapport <- as.ordered(r_df_conv$rapport)  # Ensure it's ordinal

# Weights
freqs <- table(r_df_conv$rapport)
r_df_conv$weights <- 1 / sqrt(freqs[as.character(r_df_conv$rapport)]) # Square root of inverse frequency
# r_df_conv$weights <- 1 / freqs[as.character(r_df_conv$rapport)] # Normalized inverse frequency
r_df_conv$weights <- r_df_conv$weights * nrow(r_df_conv) / sum(r_df_conv$weights)
     
# Print the unique weight values sorted by frequency
sorted_weights <- sort(table(r_df_conv$weights), decreasing = TRUE)
cat("Weights (sorted by frequency):\n")
print(sorted_weights)

# Log transform (adding 1 to avoid log(0))
raw_weights <- r_df_conv$weights
log_weights <- log1p(raw_weights)
r_df_conv$weights <- log_weights

# Print the unique weight values sorted by frequency (after log transformation)
sorted_weights <- sort(table(r_df_conv$weights), decreasing = TRUE)
cat("Weights (sorted by frequency) after log transformation:\n")
print(sorted_weights)
     
# Fit the model
clmm_model_full_other <- clmm(as.formula(formula_full_other), data = r_df_conv, weights = weights)
     
# Save the model summary to a text file        
sink("models/RQ1/weights/clmm_full_other.txt") # Redirect output to a file
print(summary(clmm_model_full_other))
sink() # Stop redirecting output
     
# Compare weighted and unweighted models
unweighted_model <- readRDS("models/RQ1/comparison/clmm_full_other_unweighted.rds")
weighted_model <- clmm_model_full_other
anova_results <- anova(unweighted_model, weighted_model)
    
# Save the comparison results to a text file
sink("models/RQ1/comparison/clmm_full_other_comparison.txt") # Redirect output to a file
print(anova_results)
sink() # Stop redirecting output
''')

#### Robustness Checks

Include age and gender of both participants.

In [ ]:
# Merge the two dataframes on 'globalPID' and 'selfPID'
df_robustness_conv = pd.merge(df_stnd_split_measures_conv, questionnaire_intake[["globalPID", "Age_part", "Sex_part"]], left_on="selfPID", right_on="globalPID", how="left")
df_robustness_conv = df_robustness_conv.drop(columns=["globalPID"])

# Rename the columns
df_robustness_conv = df_robustness_conv.rename(columns={'Age_part': 'self_age', 'Sex_part': 'self_sex'})

# Merge the two dataframes on 'globalPID' and 'otherPID'
df_robustness_conv = pd.merge(df_robustness_conv, questionnaire_intake[["globalPID", "Age_part", "Sex_part"]], left_on="otherPID", right_on="globalPID", how="left")
df_robustness_conv = df_robustness_conv.drop(columns=["globalPID"])

# Rename the columns
df_robustness_conv = df_robustness_conv.rename(columns={'Age_part': 'other_age', 'Sex_part': 'other_sex'})

# Standardize the added columns
df_robustness_conv[["self_age", "self_sex", "other_age", "other_sex"]] = standardize(df_robustness_conv[["self_age", "self_sex", "other_age", "other_sex"]])

df_robustness_conv.head()

##### Variation Inflation Factor (VIF)

Measures of both participants (excluding self_speaking_time).

In [ ]:
# Calculate VIF for self_ and other_ measures (without categorical variables)
calculate_vif(df_robustness_conv, include_prefixes=["self_", "other_"], exclude_columns=["self_recording_file", "other_recording_file", "self_speaking_time", "self_sex", "other_sex"])

##### Cumulative Link-Mixed Model (CLMM)

In [ ]:
formula_full_other_robustness = formula_full_other + ' + self_age + other_age + self_sex + other_sex'

ro.globalenv['formula_full_other_robustness'] = formula_full_other_robustness

No weights.

In [ ]:
# Enable conversion between Pandas and R data frames
pandas2ri.activate()

# Load R's 'ordinal' package
ordinal = importr("ordinal")

# Convert to R dataframe
r_df_robustness_conv = pandas2ri.py2rpy(df_robustness_conv)

# Define the CLMM model in R
ro.r.assign("r_df_robustness_conv", r_df_robustness_conv)
ro.r('''
# install.packages("ordinal") # Uncomment if you need to install the package
library(ordinal)
     
r_df_robustness_conv$rapport <- as.ordered(r_df_robustness_conv$rapport)  # Ensure it's ordinal
r_df_robustness_conv$self_sex <- as.factor(r_df_robustness_conv$self_sex) # Ensure it's categorical
r_df_robustness_conv$other_sex <- as.factor(r_df_robustness_conv$other_sex) # Ensure it's categorical

# Fit the model
clmm_model_full_other <- clmm(as.formula(formula_full_other_robustness), data = r_df_robustness_conv)
     
# Save the model summary to a text file     
sink("models/RQ1/robustness_no_weights/clmm_full_other.txt") # Redirect output to a file
print(summary(clmm_model_full_other))
sink() # Stop redirecting output
''')

Weights.

In [ ]:
# Enable conversion between Pandas and R data frames
pandas2ri.activate()

# Load R's 'ordinal' package
ordinal = importr("ordinal")

# Convert to R dataframe
r_df_robustness_conv = pandas2ri.py2rpy(df_robustness_conv)

# Define the CLMM model in R
ro.r.assign("r_df_robustness_conv", r_df_robustness_conv)
ro.r('''
# install.packages("ordinal") # Uncomment if you need to install the package
library(ordinal)
     
r_df_robustness_conv$rapport <- as.ordered(r_df_robustness_conv$rapport)  # Ensure it's ordinal
r_df_robustness_conv$self_sex <- as.factor(r_df_robustness_conv$self_sex) # Ensure it's categorical
r_df_robustness_conv$other_sex <- as.factor(r_df_robustness_conv$other_sex) # Ensure it's categorical

# Weights
freqs <- table(r_df_robustness_conv$rapport)
r_df_robustness_conv$weights <- 1 / sqrt(freqs[as.character(r_df_robustness_conv$rapport)]) # Square root of inverse frequency
# r_df_robustness_conv$weights <- 1 / freqs[as.character(r_df_robustness_conv$rapport)] # Normalized inverse frequency
r_df_robustness_conv$weights <- r_df_robustness_conv$weights * nrow(r_df_robustness_conv) / sum(r_df_robustness_conv$weights)
     
# Print the unique weight values sorted by frequency
sorted_weights <- sort(table(r_df_robustness_conv$weights), decreasing = TRUE)
cat("Weights (sorted by frequency):\n")
print(sorted_weights)

# Log transform (adding 1 to avoid log(0))
raw_weights <- r_df_robustness_conv$weights
log_weights <- log1p(raw_weights)
r_df_robustness_conv$weights <- log_weights

# Print the unique weight values sorted by frequency (after log transformation)
sorted_weights <- sort(table(r_df_robustness_conv$weights), decreasing = TRUE)
cat("Weights (sorted by frequency) after log transformation:\n")
print(sorted_weights)

# Fit the model
clmm_model_full_other <- clmm(as.formula(formula_full_other_robustness), data = r_df_robustness_conv, weights = weights)   
     
# Save the model summary to a text file     
sink("models/RQ1/robustness_weights/clmm_full_other.txt") # Redirect output to a file
print(summary(clmm_model_full_other))
sink() # Stop redirecting output
''')

### Both Settings (RQ2)

#### Correlations

Measures of one participant.

In [ ]:
correlation_matrix_self_combined = get_correlation_matrix(df_stnd_split_measures_combined, "both", True, False, False, False, name_mapping)
high_corr_pairs_self_combined = highly_correlated_pairs(correlation_matrix_self_combined)

Measures of both participants.

In [ ]:
correlation_matrix_both_combined = get_correlation_matrix(df_stnd_split_measures_combined, "both", True, True, False, False, name_mapping)
high_corr_pairs_both_both = highly_correlated_pairs(correlation_matrix_both_combined)

Main measures and interaction terms.

In [ ]:
correlation_matrix_both_combined_interact = get_correlation_matrix(df_stnd_split_measures_combined, "both", True, True, True, False, name_mapping)
high_corr_pairs_both_both = highly_correlated_pairs(correlation_matrix_both_combined_interact)

#### Variation Inflation Factor (VIF)

Measures of both participants.

In [ ]:
calculate_vif(df_stnd_split_measures_combined, include_prefixes=["self_", "other_"], include_setting=True, exclude_columns=["self_recording_file", "other_recording_file"])

Main measures and interaction terms.

In [ ]:
calculate_vif(df_stnd_split_measures_combined, include_prefixes=["self_", "other_", "setting"], exclude_columns=["self_recording_file", "other_recording_file"])

#### Remove Multicollinearity

Correlations between self_speaking_time/other_speaking_time and rapport.

In [ ]:
# Correlation between rapport and self_speaking_time and other_speaking_time
corr_rapport_self = df_stnd_split_measures_combined["self_speaking_time"].corr(df_stnd_split_measures_combined["rapport"])
corr_rapport_self_int = df_stnd_split_measures_combined["setting_x_self_speaking_time"].corr(df_stnd_split_measures_combined["rapport"])
corr_rapport_other = df_stnd_split_measures_combined["other_speaking_time"].corr(df_stnd_split_measures_combined["rapport"])
corr_rapport_other_int = df_stnd_split_measures_combined["setting_x_other_speaking_time"].corr(df_stnd_split_measures_combined["rapport"])

print(f"Correlation between self_speaking_time and rapport: {corr_rapport_self:.2f}")
print(f"Correlation between setting_x_self_speaking_time and rapport: {corr_rapport_self_int:.2f}")
print(f"Correlation between other_speaking_time and rapport: {corr_rapport_other:.2f}")
print(f"Correlation between setting_x_other_speaking_time and rapport: {corr_rapport_other_int:.2f}")

VIF after excluding self_speaking_time.

In [ ]:
calculate_vif(df_stnd_split_measures_combined, include_prefixes=["self_", "other_", "setting"], exclude_columns=["self_recording_file", "other_recording_file", "self_speaking_time", "setting_x_self_speaking_time"])

#### Modular Models

In [ ]:
calculate_vif(df_stnd_split_measures_combined, include_prefixes=["other_speaking_time", "setting_x_other_speaking_time"], include_setting=True, exclude_columns=["self_recording_file", "other_recording_file"])
calculate_vif(df_stnd_split_measures_combined, include_prefixes=["self_turn_lengths", "other_turn_lengths", "setting_x_self_turn_lengths", "setting_x_other_turn_lengths"], include_setting=True, exclude_columns=["self_recording_file", "other_recording_file"])
calculate_vif(df_stnd_split_measures_combined, include_prefixes=["self_speaking_rate", "other_speaking_rate", "setting_x_self_speaking_rate", "setting_x_other_speaking_rate"], include_setting=True, exclude_columns=["self_recording_file", "other_recording_file"])
calculate_vif(df_stnd_split_measures_combined, include_prefixes=["self_response_times", "other_response_times", "setting_x_self_response_times", "setting_x_other_response_times"], include_setting=True, exclude_columns=["self_recording_file", "other_recording_file"])

#### Cumulative Link-Mixed Model (CLMM)

Formulas.

In [ ]:
# Modular models
formula_speaking_time_interaction = (
    'rapport ~ other_speaking_time + setting_x_other_speaking_time + '
    'setting + (1 | dyadID) + (1 | selfPID)'
)

ro.globalenv['formula_speaking_time_interaction'] = formula_speaking_time_interaction

formula_turn_lengths_interaction = (
    'rapport ~ self_turn_lengths_median + setting_x_self_turn_lengths_median + '
    'self_turn_lengths_variability + setting_x_self_turn_lengths_variability + '
    'self_turn_lengths_adaptability + setting_x_self_turn_lengths_adaptability + '
    'self_turn_lengths_predictability + setting_x_self_turn_lengths_predictability + '
    'other_turn_lengths_median + setting_x_other_turn_lengths_median + '
    'other_turn_lengths_variability + setting_x_other_turn_lengths_variability + '
    'other_turn_lengths_adaptability + setting_x_other_turn_lengths_adaptability + '
    'other_turn_lengths_predictability + setting_x_other_turn_lengths_predictability + '
    'setting + (1 | dyadID) + (1 | selfPID)'
)

ro.globalenv['formula_turn_lengths_interaction'] = formula_turn_lengths_interaction

formula_speaking_rate_interaction = (
    'rapport ~ self_speaking_rate_median + setting_x_self_speaking_rate_median + '
    'self_speaking_rate_variability + setting_x_self_speaking_rate_variability + '
    'self_speaking_rate_adaptability + setting_x_self_speaking_rate_adaptability + '
    'self_speaking_rate_predictability + setting_x_self_speaking_rate_predictability + '
    'other_speaking_rate_median + setting_x_other_speaking_rate_median + '
    'other_speaking_rate_variability + setting_x_other_speaking_rate_variability + '
    'other_speaking_rate_adaptability + setting_x_other_speaking_rate_adaptability + '
    'other_speaking_rate_predictability + setting_x_other_speaking_rate_predictability + '
    'setting + (1 | dyadID) + (1 | selfPID)'
)

ro.globalenv['formula_speaking_rate_interaction'] = formula_speaking_rate_interaction

formula_response_times_interaction = (
    'rapport ~ self_response_times_median + setting_x_self_response_times_median + '
    'self_response_times_variability + setting_x_self_response_times_variability + '
    'self_response_times_adaptability + setting_x_self_response_times_adaptability + '
    'self_response_times_predictability + setting_x_self_response_times_predictability + '
    'other_response_times_median + setting_x_other_response_times_median + '
    'other_response_times_variability + setting_x_other_response_times_variability + '
    'other_response_times_adaptability + setting_x_other_response_times_adaptability + '
    'other_response_times_predictability + setting_x_other_response_times_predictability + '
    'setting + (1 | dyadID) + (1 | selfPID)'
)

ro.globalenv['formula_response_times_interaction'] = formula_response_times_interaction

# All conversation dynamics measures (except self speaking time)
formula_full_other_interaction = (
    'rapport ~ other_speaking_time + setting_x_other_speaking_time + '
    'self_turn_lengths_median + setting_x_self_turn_lengths_median + '
    'self_turn_lengths_variability + setting_x_self_turn_lengths_variability + '
    'self_turn_lengths_adaptability + setting_x_self_turn_lengths_adaptability + '
    'self_turn_lengths_predictability + setting_x_self_turn_lengths_predictability + '
    'other_turn_lengths_median + setting_x_other_turn_lengths_median + '
    'other_turn_lengths_variability + setting_x_other_turn_lengths_variability + '
    'other_turn_lengths_adaptability + setting_x_other_turn_lengths_adaptability + '
    'other_turn_lengths_predictability + setting_x_other_turn_lengths_predictability + '
    'self_speaking_rate_median + setting_x_self_speaking_rate_median + '
    'self_speaking_rate_variability + setting_x_self_speaking_rate_variability + '
    'self_speaking_rate_adaptability + setting_x_self_speaking_rate_adaptability + '
    'self_speaking_rate_predictability + setting_x_self_speaking_rate_predictability + '
    'other_speaking_rate_median + setting_x_other_speaking_rate_median + '
    'other_speaking_rate_variability + setting_x_other_speaking_rate_variability + '
    'other_speaking_rate_adaptability + setting_x_other_speaking_rate_adaptability + '
    'other_speaking_rate_predictability + setting_x_other_speaking_rate_predictability + '
    'self_response_times_median + setting_x_self_response_times_median + '
    'self_response_times_variability + setting_x_self_response_times_variability + '
    'self_response_times_adaptability + setting_x_self_response_times_adaptability + '
    'self_response_times_predictability + setting_x_self_response_times_predictability + '
    'other_response_times_median + setting_x_other_response_times_median + '
    'other_response_times_variability + setting_x_other_response_times_variability + '
    'other_response_times_adaptability + setting_x_other_response_times_adaptability + '
    'other_response_times_predictability + setting_x_other_response_times_predictability + '
    'setting + (1 | dyadID) + (1 | selfPID)'    
)

ro.globalenv['formula_full_other_interaction'] = formula_full_other_interaction

No weights.

In [ ]:
# Enable conversion between Pandas and R data frames
pandas2ri.activate()

# Load R's 'ordinal' package
ordinal = importr("ordinal")

# Convert to R dataframe
r_df_combined = pandas2ri.py2rpy(df_stnd_split_measures_combined)

# Define the CLMM model in R
ro.r.assign("r_df_combined", r_df_combined)
ro.r('''
# install.packages("ordinal") # Uncomment if you need to install the package
library(ordinal)
     
r_df_combined$rapport <- as.ordered(r_df_combined$rapport)  # Ensure it's ordinal

# Fit the models
clmm_model_speaking_time <- clmm(as.formula(formula_speaking_time_interaction), data = r_df_combined)
clmm_model_turn_lengths <- clmm(as.formula(formula_turn_lengths_interaction), data = r_df_combined)
clmm_model_speaking_rate <- clmm(as.formula(formula_speaking_rate_interaction), data = r_df_combined)
clmm_model_response_times <- clmm(as.formula(formula_response_times_interaction), data = r_df_combined)
clmm_model_full_other <- clmm(as.formula(formula_full_other_interaction), data = r_df_combined)
     
# Save the model summaries to text files
sink("models/RQ2/no_weights/clmm_speaking_time.txt") # Redirect output to a file
print(summary(clmm_model_speaking_time))
sink() # Stop redirecting output
     
sink("models/RQ2/no_weights/clmm_turn_lengths.txt") # Redirect output to a file
print(summary(clmm_model_turn_lengths))
sink() # Stop redirecting output
     
sink("models/RQ2/no_weights/clmm_speaking_rate.txt") # Redirect output to a file
print(summary(clmm_model_speaking_rate))
sink() # Stop redirecting output
     
sink("models/RQ2/no_weights/clmm_response_times.txt") # Redirect output to a file
print(summary(clmm_model_response_times))
sink() # Stop redirecting output
     
sink("models/RQ2/no_weights/clmm_full_other.txt") # Redirect output to a file
print(summary(clmm_model_full_other))
sink() # Stop redirecting output
     
# Save the full_other model to a file
saveRDS(clmm_model_full_other, file = "models/RQ2/comparison/clmm_full_other_unweighted.rds")
''')

Weights.

In [ ]:
# Enable conversion between Pandas and R data frames
pandas2ri.activate()

# Load R's 'ordinal' package
ordinal = importr("ordinal")

# Convert to R dataframe
r_df_combined = pandas2ri.py2rpy(df_stnd_split_measures_combined)

# Define the CLMM model in R
ro.r.assign("r_df_combined", r_df_combined)
ro.r('''
# install.packages("ordinal") # Uncomment if you need to install the package
library(ordinal)
     
r_df_combined$rapport <- as.ordered(r_df_combined$rapport)  # Ensure it's ordinal
     
# Weights
freqs <- table(r_df_combined$rapport)
r_df_combined$weights <- 1 / sqrt(freqs[as.character(r_df_combined$rapport)]) # Square root of inverse frequency
# r_df_combined$weights <- 1 / freqs[as.character(r_df_combined$rapport)] # Normalized inverse frequency
r_df_combined$weights <- r_df_combined$weights * nrow(r_df_combined) / sum(r_df_combined$weights)
     
# Print the unique weight values sorted by frequency
sorted_weights <- sort(table(r_df_combined$weights), decreasing = TRUE)
cat("Weights (sorted by frequency):\n")
print(sorted_weights)

# Log transform (adding 1 to avoid log(0))
raw_weights <- r_df_combined$weights
log_weights <- log1p(raw_weights)
r_df_combined$weights <- log_weights

# Print the unique weight values sorted by frequency (after log transformation)
sorted_weights <- sort(table(r_df_combined$weights), decreasing = TRUE)
cat("Weights (sorted by frequency) after log transformation:\n")
print(sorted_weights)

# Fit the models
clmm_model_speaking_time <- clmm(as.formula(formula_speaking_time_interaction), data = r_df_combined, weights = weights)
clmm_model_turn_lengths <- clmm(as.formula(formula_turn_lengths_interaction), data = r_df_combined, weights = weights)
clmm_model_speaking_rate <- clmm(as.formula(formula_speaking_rate_interaction), data = r_df_combined, weights = weights)
clmm_model_response_times <- clmm(as.formula(formula_response_times_interaction), data = r_df_combined, weights = weights)
clmm_model_full_other <- clmm(as.formula(formula_full_other_interaction), data = r_df_combined, weights = weights)
     
# Save the model summaries to text files
sink("models/RQ2/weights/clmm_speaking_time.txt") # Redirect output to a file
print(summary(clmm_model_speaking_time))
sink() # Stop redirecting output
     
sink("models/RQ2/weights/clmm_turn_lengths.txt") # Redirect output to a file
print(summary(clmm_model_turn_lengths))
sink() # Stop redirecting output
     
sink("models/RQ2/weights/clmm_speaking_rate.txt") # Redirect output to a file
print(summary(clmm_model_speaking_rate))
sink() # Stop redirecting output
     
sink("models/RQ2/weights/clmm_response_times.txt") # Redirect output to a file
print(summary(clmm_model_response_times))
sink() # Stop redirecting output
     
sink("models/RQ2/weights/clmm_full_other.txt") # Redirect output to a file
print(summary(clmm_model_full_other))
sink() # Stop redirecting output
     
# Compare weighted and unweighted models
unweighted_model <- readRDS("models/RQ2/comparison/clmm_full_other_unweighted.rds")
weighted_model <- clmm_model_full_other
anova_results <- anova(unweighted_model, weighted_model)
     
# Save the comparison results to a text file
sink("models/RQ2/comparison/clmm_full_other_comparison.txt") # Redirect output to a file
print(anova_results)
sink() # Stop redirecting output
''')

#### Robustness Checks

Include age and gender of both participants

In [ ]:
# Merge the two dataframes on 'globalPID' and 'selfPID'
df_robustness_combined = pd.merge(df_stnd_split_measures_combined, questionnaire_intake[["globalPID", "Age_part", "Sex_part"]], left_on="selfPID", right_on="globalPID", how="left")
df_robustness_combined = df_robustness_combined.drop(columns=["globalPID"])

# Rename the columns
df_robustness_combined = df_robustness_combined.rename(columns={'Age_part': 'self_age', 'Sex_part': 'self_sex'})

# Merge the two dataframes on 'globalPID' and 'otherPID'
df_robustness_combined = pd.merge(df_robustness_combined, questionnaire_intake[["globalPID", "Age_part", "Sex_part"]], left_on="otherPID", right_on="globalPID", how="left")
df_robustness_combined = df_robustness_combined.drop(columns=["globalPID"])

# Rename the columns
df_robustness_combined = df_robustness_combined.rename(columns={'Age_part': 'other_age', 'Sex_part': 'other_sex'})

# Standardize the added columns
df_robustness_combined[["self_age", "self_sex", "other_age", "other_sex"]] = standardize(df_robustness_combined[["self_age", "self_sex", "other_age", "other_sex"]])

df_robustness_combined.head()

##### Variation Inflation Factor (VIF)

Measures of both participants (excluding self_speaking_time).

In [ ]:
# Calculate VIF for self_ and other_ measures (without categorical variables)
calculate_vif(df_robustness_combined, include_prefixes=["self_", "other_"], include_setting=True, exclude_columns=["self_recording_file", "other_recording_file", "self_speaking_time", "self_sex", "other_sex"])

Main measures and interaction terms.

In [ ]:
calculate_vif(df_robustness_combined, include_prefixes=["self_", "other_", "setting"], exclude_columns=["self_recording_file", "other_recording_file", "self_speaking_time", "setting_x_self_speaking_time", "self_sex", "other_sex"])

Modular models.

In [ ]:
calculate_vif(df_robustness_combined, include_prefixes=["other_speaking_time", "setting_x_other_speaking_time", "self_age", "other_age"], include_setting=True, exclude_columns=["self_recording_file", "other_recording_file", "self_sex", "other_sex"])
calculate_vif(df_robustness_combined, include_prefixes=["self_turn_lengths", "other_turn_lengths", "setting_x_self_turn_lengths", "setting_x_other_turn_lengths", "self_age", "other_age"], include_setting=True, exclude_columns=["self_recording_file", "other_recording_file", "self_sex", "other_sex"])
calculate_vif(df_robustness_combined, include_prefixes=["self_speaking_rate", "other_speaking_rate", "setting_x_self_speaking_rate", "setting_x_other_speaking_rate", "self_age", "other_age"], include_setting=True, exclude_columns=["self_recording_file", "other_recording_file", "self_sex", "other_sex"])
calculate_vif(df_robustness_combined, include_prefixes=["self_response_times", "other_response_times", "setting_x_self_response_times", "setting_x_other_response_times", "self_age", "other_age"], include_setting=True, exclude_columns=["self_recording_file", "other_recording_file", "self_sex", "other_sex"])

##### Cumulative Link-Mixed Model (CLMM)

Formulas.

In [ ]:
formula_speaking_time_interaction_robustness = formula_speaking_time_interaction + ' + self_age + other_age + self_sex + other_sex'
ro.globalenv['formula_speaking_time_interaction_robustness'] = formula_speaking_time_interaction_robustness

formula_turn_lengths_interaction_robustness = formula_turn_lengths_interaction + ' + self_age + other_age + self_sex + other_sex'
ro.globalenv['formula_turn_lengths_interaction_robustness'] = formula_turn_lengths_interaction_robustness

formula_speaking_rate_interaction_robustness = formula_speaking_rate_interaction + ' + self_age + other_age + self_sex + other_sex'
ro.globalenv['formula_speaking_rate_interaction_robustness'] = formula_speaking_rate_interaction_robustness

formula_response_times_interaction_robustness = formula_response_times_interaction + ' + self_age + other_age + self_sex + other_sex'
ro.globalenv['formula_response_times_interaction_robustness'] = formula_response_times_interaction_robustness

formula_full_other_interaction_robustness = formula_full_other_interaction + ' + self_age + other_age + self_sex + other_sex'
ro.globalenv['formula_full_other_interaction_robustness'] = formula_full_other_interaction_robustness

No weights.

In [ ]:
# Enable conversion between Pandas and R data frames
pandas2ri.activate()

# Load R's 'ordinal' package
ordinal = importr("ordinal")

# Convert to R dataframe
r_df_robustness_combined = pandas2ri.py2rpy(df_robustness_combined)

# Define the CLMM model in R
ro.r.assign("df_robustness_combined", df_robustness_combined)
ro.r('''
# install.packages("ordinal") # Uncomment if you need to install the package
library(ordinal)
     
df_robustness_combined$rapport <- as.ordered(df_robustness_combined$rapport)  # Ensure it's ordinal
df_robustness_combined$self_sex <- as.factor(df_robustness_combined$self_sex) # Ensure it's categorical
df_robustness_combined$other_sex <- as.factor(df_robustness_combined$other_sex) # Ensure it's categorical

# Fit the models
clmm_model_speaking_time <- clmm(as.formula(formula_speaking_time_interaction_robustness), data = df_robustness_combined)
clmm_model_turn_lengths <- clmm(as.formula(formula_turn_lengths_interaction_robustness), data = df_robustness_combined)
clmm_model_speaking_rate <- clmm(as.formula(formula_speaking_rate_interaction_robustness), data = df_robustness_combined)
clmm_model_response_times <- clmm(as.formula(formula_response_times_interaction_robustness), data = df_robustness_combined)
clmm_model_full_other <- clmm(as.formula(formula_full_other_interaction_robustness), data = df_robustness_combined)
     
# Save the model summaries to text files
sink("models/RQ2/robustness_no_weights/clmm_speaking_time.txt") # Redirect output to a file
print(summary(clmm_model_speaking_time))
sink() # Stop redirecting output
     
sink("models/RQ2/robustness_no_weights/clmm_turn_lengths.txt") # Redirect output to a file
print(summary(clmm_model_turn_lengths))
sink() # Stop redirecting output
     
sink("models/RQ2/robustness_no_weights/clmm_speaking_rate.txt") # Redirect output to a file
print(summary(clmm_model_speaking_rate))
sink() # Stop redirecting output
     
sink("models/RQ2/robustness_no_weights/clmm_response_times.txt") # Redirect output to a file
print(summary(clmm_model_response_times))
sink() # Stop redirecting output
     
sink("models/RQ2/robustness_no_weights/clmm_full_other.txt") # Redirect output to a file
print(summary(clmm_model_full_other))
sink() # Stop redirecting output
''')

Weights.

In [ ]:
# Enable conversion between Pandas and R data frames
pandas2ri.activate()

# Load R's 'ordinal' package
ordinal = importr("ordinal")

# Convert to R dataframe
r_df_robustness_combined = pandas2ri.py2rpy(df_robustness_combined)

# Define the CLMM model in R
ro.r.assign("r_df_robustness_combined", r_df_robustness_combined)
ro.r('''
# install.packages("ordinal") # Uncomment if you need to install the package
library(ordinal)
     
r_df_robustness_combined$rapport <- as.ordered(r_df_robustness_combined$rapport)  # Ensure it's ordinal
r_df_robustness_combined$self_sex <- as.factor(r_df_robustness_combined$self_sex) # Ensure it's categorical
r_df_robustness_combined$other_sex <- as.factor(r_df_robustness_combined$other_sex) # Ensure it's categorical

# Weights
freqs <- table(r_df_robustness_combined$rapport)
r_df_robustness_combined$weights <- 1 / sqrt(freqs[as.character(r_df_robustness_combined$rapport)]) # Square root of inverse frequency
# r_df_robustness_combined$weights <- 1 / freqs[as.character(r_df_robustness_combined$rapport)] # Normalized inverse frequency
r_df_robustness_combined$weights <- r_df_robustness_combined$weights * nrow(r_df_robustness_combined) / sum(r_df_robustness_combined$weights)
     
# Print the unique weight values sorted by frequency
sorted_weights <- sort(table(r_df_robustness_combined$weights), decreasing = TRUE)
cat("Weights (sorted by frequency):\n")
print(sorted_weights)

# Log transform (adding 1 to avoid log(0))
raw_weights <- r_df_robustness_combined$weights
log_weights <- log1p(raw_weights)
r_df_robustness_combined$weights <- log_weights

# Print the unique weight values sorted by frequency (after log transformation)
sorted_weights <- sort(table(r_df_robustness_combined$weights), decreasing = TRUE)
cat("Weights (sorted by frequency) after log transformation:\n")
print(sorted_weights)

# Fit the models
clmm_model_speaking_time <- clmm(as.formula(formula_speaking_time_interaction_robustness), data = r_df_robustness_combined, weights = weights)
clmm_model_turn_lengths <- clmm(as.formula(formula_turn_lengths_interaction_robustness), data = r_df_robustness_combined, weights = weights)
clmm_model_speaking_rate <- clmm(as.formula(formula_speaking_rate_interaction_robustness), data = r_df_robustness_combined, weights = weights)
clmm_model_response_times <- clmm(as.formula(formula_response_times_interaction_robustness), data = r_df_robustness_combined, weights = weights)
clmm_model_full_other <- clmm(as.formula(formula_full_other_interaction_robustness), data = r_df_robustness_combined, weights = weights)
     
# Save the model summaries to text files
sink("models/RQ2/robustness_weights/clmm_speaking_time.txt") # Redirect output to a file
print(summary(clmm_model_speaking_time))
sink() # Stop redirecting output
     
sink("models/RQ2/robustness_weights/clmm_turn_lengths.txt") # Redirect output to a file
print(summary(clmm_model_turn_lengths))
sink() # Stop redirecting output
     
sink("models/RQ2/robustness_weights/clmm_speaking_rate.txt") # Redirect output to a file
print(summary(clmm_model_speaking_rate))
sink() # Stop redirecting output
     
sink("models/RQ2/robustness_weights/clmm_response_times.txt") # Redirect output to a file
print(summary(clmm_model_response_times))
sink() # Stop redirecting output
     
sink("models/RQ2/robustness_weights/clmm_full_other.txt") # Redirect output to a file
print(summary(clmm_model_full_other))
sink() # Stop redirecting output
''')